In [1]:
import mlflow
mlflow.set_tracking_uri("http://ec2-3-82-21-172.compute-1.amazonaws.com:5000/")

In [2]:
mlflow.set_experiment("Exp 5 - ML Algos with HP Tuning")

2026/01/05 16:16:16 INFO mlflow.tracking.fluent: Experiment with name 'Exp 5 - ML Algos with HP Tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflowexperiment-26/5', creation_time=1767609976575, experiment_id='5', last_update_time=1767609976575, lifecycle_stage='active', name='Exp 5 - ML Algos with HP Tuning', tags={}>

In [3]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [6]:
df = pd.read_csv('reddit_preprocessing.csv').dropna()
df.shape

(36662, 2)

In [7]:
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

df = df.dropna(subset=['category'])

ngram_range = (1, 3) 
max_features = 10000 

X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train) 
X_test_vec = vectorizer.transform(X_test) 

smote = SMOTE(random_state=42)
X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)

def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        mlflow.log_param("algo_name", model_name)

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        mlflow.sklearn.log_model(model, f"{model_name}_model")

def objective_xgboost(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    model = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=42)
    return accuracy_score(y_test, model.fit(X_train_vec, y_train).predict(X_test_vec))


def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_xgboost, n_trials=30)

    best_params = study.best_params
    best_model = XGBClassifier(n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'], max_depth=best_params['max_depth'], random_state=42)

    log_mlflow("XGBoost", best_model, X_train_vec, X_test_vec, y_train, y_test)

run_optuna_experiment()


[I 2026-01-05 16:20:41,852] A new study created in memory with name: no-name-2091e281-906a-455a-b7f9-65e82e3d47b4
[I 2026-01-05 16:22:23,673] Trial 0 finished with value: 0.6090276830765035 and parameters: {'n_estimators': 196, 'learning_rate': 0.008743150504877693, 'max_depth': 4}. Best is trial 0 with value: 0.6090276830765035.
[I 2026-01-05 16:24:16,410] Trial 1 finished with value: 0.575617073503341 and parameters: {'n_estimators': 191, 'learning_rate': 0.003350551749102348, 'max_depth': 4}. Best is trial 0 with value: 0.6090276830765035.
[I 2026-01-05 16:25:24,084] Trial 2 finished with value: 0.5236601663711987 and parameters: {'n_estimators': 111, 'learning_rate': 0.00016636073745254507, 'max_depth': 4}. Best is trial 0 with value: 0.6090276830765035.
[I 2026-01-05 16:42:04,878] Trial 3 finished with value: 0.6123005591163234 and parameters: {'n_estimators': 172, 'learning_rate': 0.00026449509587946583, 'max_depth': 10}. Best is trial 3 with value: 0.6123005591163234.
[I 2026-01

KeyboardInterrupt: 